In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

#import CNNConfig as Conf

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

#------------------------------------------

ScratchFileTrain="Scratch.csv"
inputfolderTrain='./canny/'

ScratchFileTest="Scratchtest.csv"
inputfolderTest='./cannytest/'

targetinputshape=(100,100)

#------------------------------------------
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(*targetinputshape,1)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
#------------------------------------------
learningrate=0.01
batch_size=64
epochs=150

In [24]:
from PIL import Image,ImageOps
import numpy as np
import glob

def filetoarray(imagenum,inputfolder='./canny'):
    from keras.preprocessing.image import load_img
    from keras.preprocessing.image import img_to_array
    from keras.preprocessing.image import array_to_img
    img = load_img(inputfolder+'/sensorphoto-'+str(imagenum)+'-1.BMP.png',color_mode="grayscale",target_size=targetinputshape)
    img_array = img_to_array(img)
    return img_array

In [25]:
df=pd.read_csv(ScratchFileTrain)
df['ImageLoc'] = df.ImageLoc.astype(int)
df['array'] = df.apply(lambda row: filetoarray(row.ImageLoc,inputfolderTrain), axis=1)

dftest=pd.read_csv(ScratchFileTest)
dftest['ImageLoc'] = dftest.ImageLoc.astype(int)
dftest['array'] = dftest.apply(lambda row: filetoarray(row.ImageLoc,inputfolderTest), axis=1)

In [26]:
df=df.query("Target!=2")
dftest=dftest.query("Target!=2")

In [27]:
ss=len(df.query("Target==1"))

In [28]:
bb=len(df.query("Target==0"))

In [29]:
#traindf,testdf=train_test_split(df,test_size=0.3,random_state=42)
#df['Wt']=1
#df.loc[df['Target']==0,"Wt"]=ss / bb
#df.loc[df['Target']==1,"Wt"]=1

class_weight = {0: ss/bb,
                1: 1.}

X_train=np.array(list(df["array"]))
Y_train=df["Target"]
Y_train = to_categorical(Y_train)
#Wt_train = df["Wt"]

X_test=np.array(list(dftest["array"]))
Y_test=dftest["Target"]
Y_test = to_categorical(Y_test)

In [30]:
print("Train image shape: "+str(X_train.shape))
#print("Test image shape: "+str(X_test.shape))

Train image shape: (87, 100, 100, 1)


In [31]:
from tensorflow.keras import optimizers
from tensorflow.keras.losses import categorical_crossentropy
model.compile(loss=categorical_crossentropy,
              optimizer=optimizers.Adam(lr=learningrate),
              metrics=['accuracy'])

trainhistory=model.fit(X_train, Y_train,batch_size=batch_size,epochs=epochs,verbose=1,class_weight=class_weight)

Epoch 1/150
87/87 [==============================] - 1s 10ms/sample - loss: 311.3339 - acc: 0.4483
Epoch 2/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0991 - acc: 0.2069
Epoch 3/150
87/87 [==============================] - 1s 10ms/sample - loss: 1.0993 - acc: 0.7931
Epoch 4/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.1002 - acc: 0.7931
Epoch 5/150
87/87 [==============================] - 1s 10ms/sample - loss: 1.1000 - acc: 0.7931
Epoch 6/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0998 - acc: 0.7931
Epoch 7/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0998 - acc: 0.7931
Epoch 8/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0997 - acc: 0.7931
Epoch 9/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0999 - acc: 0.7931
Epoch 10/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.1000 - acc: 0.7931
Epoch 11/150
87/87 [==============

87/87 [==============================] - 1s 9ms/sample - loss: 1.0995 - acc: 0.7931
Epoch 86/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0995 - acc: 0.7931
Epoch 87/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0995 - acc: 0.7931
Epoch 88/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0996 - acc: 0.7931
Epoch 89/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0995 - acc: 0.7931
Epoch 90/150
87/87 [==============================] - 1s 10ms/sample - loss: 1.0995 - acc: 0.7931
Epoch 91/150
87/87 [==============================] - 1s 10ms/sample - loss: 1.0996 - acc: 0.7931
Epoch 92/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0999 - acc: 0.7931
Epoch 93/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0994 - acc: 0.7931
Epoch 94/150
87/87 [==============================] - 1s 9ms/sample - loss: 1.0996 - acc: 0.7931
Epoch 95/150
87/87 [=====================

In [ ]:
score = model.evaluate(X_train, Y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])